In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./train_Numerical_data.csv')

In [3]:
df.head()

,Store_id,Item_id,Date,Quantity,Price,Net_Price,Net_Amount,Discount_Amount,Cost_Amount,Customer_Discount,Total_Discount,Periodic_Discount,Refund Qty_,date
0,S_6,Item_16,1532304000000000000,0.125,2.0,1.869,-1.869,0.0,-0.182,0.0,0.0,0.0,1.0,2018-07-23
1,S_6,Item_16,1532390400000000000,0.125,2.0,1.869,-1.870,0.0,-0.182,0.0,0.0,0.0,1.0,2018-07-24
2,S_6,Item_16,1532563200000000000,0.125,2.0,1.869,-1.869,0.0,-0.182,0.0,0.0,0.0,1.0,2018-07-26
3,S_6,Item_16,1532563200000000000,0.125,2.0,1.869,-1.869,0.0,-0.182,0.0,0.0,0.0,1.0,2018-07-26
4,S_6,Item_16,1532649600000000000,0.125,2.0,1.869,-1.869,0.0,-0.182,0.0,0.0,0.0,1.0,2018-07-27


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5451442 entries, 0 to 5451441
Data columns (total 14 columns):
Store_id             object
Item_id              object
Date                 int64
Quantity             float64
Price                float64
Net_Price            float64
Net_Amount           float64
Discount_Amount      float64
Cost_Amount          float64
Customer_Discount    float64
Total_Discount       float64
Periodic_Discount    float64
Refund Qty_          float64
date                 object
dtypes: float64(10), int64(1), object(3)
memory usage: 582.3+ MB


In [5]:
df.isna().sum()

Store_id             0
Item_id              0
Date                 0
Quantity             0
Price                0
Net_Price            0
Net_Amount           0
Discount_Amount      0
Cost_Amount          0
Customer_Discount    0
Total_Discount       0
Periodic_Discount    0
Refund Qty_          0
date                 0
dtype: int64

In [6]:
df.isnull().sum()

Store_id             0
Item_id              0
Date                 0
Quantity             0
Price                0
Net_Price            0
Net_Amount           0
Discount_Amount      0
Cost_Amount          0
Customer_Discount    0
Total_Discount       0
Periodic_Discount    0
Refund Qty_          0
date                 0
dtype: int64

In [7]:
def memory_usage_per_type(data_frame):
    types = ['float','int','object']
    for type in types:
        selected_col=data_frame.select_dtypes(include=[type])
        memory_usage_of_selected_type_b = selected_col.memory_usage(deep=True).sum()
        memory_usage_of_selected_type_mb=memory_usage_of_selected_type_b/1024**2
        print("memory usage for {} columns: {:03.2f} MB".format(type,memory_usage_of_selected_type_mb))


In [8]:
memory_usage_per_type(df)

memory usage for float columns: 415.91 MB
memory usage for int columns: 0.00 MB
memory usage for object columns: 1007.46 MB


In [9]:
def memory_usage(data_frame):
    return data_frame.memory_usage(deep=True).sum()/1024**2

In [10]:
print('Memory use before optimization {:.2f} MB'.format(memory_usage(df)))

Memory use before optimization 1464.96 MB


### limit of subtypes

In [11]:
def subtypes_limit(subtype):
    if "int" in subtype:
        return np.iinfo(subtype).min,np.iinfo(subtype).max
    if "float" in subtype:
        return np.finfo(subtype).min,np.finfo(subtype).max

In [12]:
for type in ['int8','float16','int32','int64','uint8','uint16','uint32','uint64']:
    print(subtypes_limit(type))

(-128, 127)
(-65500.0, 65500.0)
(-2147483648, 2147483647)
(-9223372036854775808, 9223372036854775807)
(0, 255)
(0, 65535)
(0, 4294967295)
(0, 18446744073709551615)


### Downcast float and int column 

In [13]:
#Solution 1
def downcast_and_compare_memory_consuming_by_type_first(type_):
    selected_type = df.select_dtypes(include=[type_])
    converted_type = selected_type.apply(pd.to_numeric,downcast=type_)
    memory_before = memory_usage(selected_type)
    memory_after = memory_usage(converted_type)
    print('before the downcast:{:.2f} MB'.format(memory_before))
    print('after the downcast:{:.2f} MB'.format(memory_after))
    print('')
    compare_type_changes = pd.concat([selected_type.dtypes,converted_type.dtypes],axis=1)
    compare_type_changes.columns = ['before','after']
    return compare_type_changes.apply(pd.Series.value_counts)
    

In [14]:
downcast_and_compare_memory_consuming_by_type_first('float')

before the downcast:415.91 MB
after the downcast:207.96 MB



,before,after
float32,NaN,10.0
float64,10.0,NaN


In [15]:
#Solution 2
def downcast_and_compare_memory_consuming_by_type_second(type_):
    selected_col = df.select_dtypes(include=[type_])
    memory_before = memory_usage(selected_col)
    optimize_col = selected_col.copy()
    for col in optimize_col.columns:
        mx_col = optimize_col[col].max()
        mn_col = optimize_col[col].min()
        if type_=='int':
            if mn_col>0:
                    if mx_col < 255:
                        optimize_col[col] = optimize_col[col].astype(np.uint8)
                    elif mx_col < 65535:
                        optimize_col[col] = optimize_col[col].astype(np.uint16)
                    elif mx_col < 4294967295:
                        optimize_col[col] = optimize_col[col].astype(np.uint32)
                    else:
                        optimize_col[col] = optimize_col[col].astype(np.uint64)
            else:
                    if mn_col > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        optimize_col[col] = optimize_col[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        optimize_col[col] = optimize_col[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        optimize_col[col] = optimize_col[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        optimize_col[col] = optimize_col[col].astype(np.int64)
        else:
            if mn_col > np.finfo(np.float16).min and mx_col < np.finfo(np.float16).max:
                    optimize_col[col] = optimize_col[col].astype(np.float16)
            elif mn_col > np.finfo(np.float32).min and mx_col < np.finfo(np.float32).max:
                    optimize_col[col] = optimize_col[col].astype(np.float32)
            else:
                    optimize_col[col] = optimize_col[col].astype(np.float64)
            
    memory_after = memory_usage(optimize_col)
    print('before the downcast:{:.2f} MB'.format(memory_before))
    print('after the downcast:{:.2f} MB'.format(memory_after))
    compare_type_changes = pd.concat([selected_col.dtypes,optimize_col.dtypes],axis=1)
    compare_type_changes.columns = ['before','after']
    return compare_type_changes.apply(pd.Series.value_counts)

In [16]:
downcast_and_compare_memory_consuming_by_type_second('float')

before the downcast:415.91 MB
after the downcast:103.98 MB


,before,after
float16,NaN,10.0
float64,10.0,NaN


### Downcast the date object 

In [17]:
'object' in str(df['date'].dtypes)

True

In [18]:
df_op = df.copy()
df_op['date'] = pd.to_datetime(df_op['date'],format='%Y-%m-%d')
memory_before = memory_usage(df)
memory_after = memory_usage(df_op)
print('before : {:.2f} MB'.format(memory_before))
print('after : {:.2f} MB'.format(memory_after))

before : 1464.96 MB
after : 1158.23 MB


### Downcast the object data

In [19]:
def downcast_object():
    selected_col = df[['Store_id','Item_id']]
    print('before the downcast:{:.2f} MB'.format(memory_usage(selected_col)))
    optimize_col = selected_col.copy()
    for col in optimize_col.columns:
        numbr_of_unique = len(optimize_col[col].unique())
        numbr_total = len(optimize_col[col])
        if numbr_of_unique/numbr_total<0.5:
            
            optimize_col[col] = optimize_col[col].astype('category')
    print('before the downcast:{:.2f} MB'.format(memory_usage(optimize_col)))
    

In [20]:
downcast_object()

before the downcast:659.13 MB
before the downcast:15.66 MB


### Downcast all the date now 

In [21]:
def reduce_memory(data_frame):
    memory_before = memory_usage(data_frame)
    print('before: {:.2f} MB'.format(memory_before))
    subtype_int = ['uint8','uint16','uint32','uint64','int8','int16','int32','int64']
    subtype_float = ['float16','float32','float64']
    for col in data_frame.columns:
        col_type = str(data_frame[col].dtypes)
        mx_col = data_frame[col].max()
        mn_col = data_frame[col].min()
        if 'int'in col_type:
            for ele in subtype_int:
                if mn_col>np.iinfo(ele).min and mx_col<np.iinfo(ele).max:
                    data_frame[col] = data_frame[col].astype(ele)
                    break
        
        elif 'float' in col_type:
            for ele in subtype_float:
                if mn_col>np.finfo(ele).min and mx_col<np.finfo(ele).max:
                    data_frame[col] = data_frame[col].astype(ele)
                    break  
        elif 'object' in col_type:
                if col=='date':
                    data_frame['date'] = pd.to_datetime(data_frame['date'],format='%Y-%m-%d')
                else:
                    numbr_of_unique = len(df[col].unique())
                    numbr_total = len(df[col])
                    if numbr_of_unique/numbr_total<0.5:
                        
                        df[col] = df[col].astype('category')
    memory_after = memory_usage(data_frame)
    print('after:{:.2f} MB'.format(memory_after))
    print('Decreased by: {:.2f} % '.format(100*(memory_before-memory_after)/memory_before))

In [22]:
reduce_memory(df)

before: 1464.96 MB
after:202.82 MB
Decreased by: 86.16 % 
